In [1]:
import os
import math
import subprocess
import sys
import re

In [2]:
# Function to read a pdb file and extract HETATM 1 atom coordinates
def read_pocket_pdb(file_path):
    atom_data = []
    with open(file_path, 'r') as file:
        for line in file:
            # Only read lines that start with HETATM and atom number is 1
            if line.startswith("HETATM") and line[24:27].strip() == "1":
                try:
                    # Extracting the x, y, z coordinates from fixed positions
                    x = float(line[30:38].strip())
                    y = float(line[38:46].strip())
                    z = float(line[46:54].strip())
                    atom_data.append([x, y, z])
                except ValueError:
                    # In case of a conversion issue, we skip the line
                    continue
    return atom_data

In [3]:

def get_pocket_dimensions(atom_data):
    min_coords = [min(coords[i] for coords in atom_data) for i in range(3)]
    max_coords = [max(coords[i] for coords in atom_data) for i in range(3)]
    center = [math.ceil((max_coords[i] + min_coords[i]) / 2) for i in range(3)]
    size = [math.ceil(max_coords[i] - min_coords[i] + 5) for i in range(3)]
    return center, size

In [4]:
def process_fpocket_output(pocket_path):
    
            pdb_file = os.path.join(pocket_path, f"{pocket_path.split('/')[-1]}.pdb")
            if os.path.isfile(pdb_file):
                atom_data = read_pocket_pdb(pdb_file)
                center,size = get_pocket_dimensions(atom_data)
                # Here you can do something with the min_coords and max_coords
                print(f"Processed: {pdb_file}")
                return center

In [5]:
# 首先进行pocket
pdbs_dir = '/mnt/e/SDT/output/CORASON/modify_fabB1_fadb_nr_rep_0.7_0.7_rep_seq/pdb/'

#生成口袋
for pro_pdb in os.listdir(pdbs_dir):
    if pro_pdb.endswith('.pdb'):
        # 生成pocket文件
        pro_pdb_path = os.path.join(pdbs_dir,pro_pdb)
        command = f"fpocket -f {pro_pdb_path}"
        try:
            subprocess.run(command, shell=True, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.PIPE, text=True)
            
        except subprocess.CalledProcessError as e:
            print(f"Error occurred while running command: {e.cmd}")
            print(f"Error output: {e.stderr}")

In [8]:

pdbs_dir = '/mnt/e/SDT/output/CORASON/modify_fabB1_fadb_nr_rep_0.7_0.7_rep_seq/pdb/'
mols_dir = '/mnt/e/SDT/output/CORASON/modify_fabB1_fadb_nr_rep_0.7_0.7_rep_seq/mols/'
vina_path = '/mnt/e/Vina-GPU-2.0-main/Vina-GPU+/input_file_example'
for pro_pdbqt in os.listdir(pdbs_dir):
    if pro_pdbqt.endswith('pdbqt'):
        # 口袋文件路径
        pro_pocket_path = os.path.join(pdbs_dir,pro_pdbqt.replace('.pdbqt','_out'))
        
        # 解析口袋坐标
        center = process_fpocket_output(pro_pocket_path)

        # 生成对接文件
        win_pro_dir = "E:\SDT\output\CORASON\modify_fabB1_fadb_nr_rep_0.7_0.7_rep_seq\pdb\\"
        
        if center is not None:
                    
            with open(os.path.join(vina_path,pro_pdbqt.replace('.pdbqt','.config')), 'w') as f:
                f.write(f"receptor = {win_pro_dir}{pro_pdbqt}\n")
                f.write(f"ligand_directory = E:\SDT\output\CORASON\modify_fabB1_fadb_nr_rep_0.7_0.7_rep_seq\mols\{pro_pdbqt.replace('.pdbqt','')}\n")
                f.write(f"center_x = {center[0]}\n")
                f.write(f"center_y = {center[1]}\n")
                f.write(f"center_z = {center[2]}\n")
                f.write(f"size_x = 30\n")
                f.write(f"size_y = 30\n")
                f.write(f"size_z = 30\n")
                f.write(f"thread = 10000\n")
                # f.write(f"out = E:\SDT\output\CORASON\modify_fabB1_fadb_nr_rep_0.7_0.7_rep_seq\Vinaoutput\{pro_pdbqt.replace('pdbqt','')}_{mols}\n")
                f.write(f"num_modes=1\n")
                f.write(f'search_depth = 1')
        else:
            print(f"No pocket found for {pro_pdbqt}")
# 生成执行脚本
for config_txt in os.listdir(vina_path):
    if config_txt.endswith('.config'):
        with open ('/mnt/e/Vina-GPU-2.0-main/Vina-GPU+/run_vina.sh','a') as f:
            print(f'./Vina-GPU+.exe --config=./input_file_example/{config_txt}',file=f)

Processed: /mnt/e/SDT/output/CORASON/modify_fabB1_fadb_nr_rep_0.7_0.7_rep_seq/pdb/0_out/0_out.pdb
Processed: /mnt/e/SDT/output/CORASON/modify_fabB1_fadb_nr_rep_0.7_0.7_rep_seq/pdb/1_out/1_out.pdb
Processed: /mnt/e/SDT/output/CORASON/modify_fabB1_fadb_nr_rep_0.7_0.7_rep_seq/pdb/10_out/10_out.pdb
Processed: /mnt/e/SDT/output/CORASON/modify_fabB1_fadb_nr_rep_0.7_0.7_rep_seq/pdb/100_out/100_out.pdb
Processed: /mnt/e/SDT/output/CORASON/modify_fabB1_fadb_nr_rep_0.7_0.7_rep_seq/pdb/101_out/101_out.pdb
Processed: /mnt/e/SDT/output/CORASON/modify_fabB1_fadb_nr_rep_0.7_0.7_rep_seq/pdb/102_out/102_out.pdb
Processed: /mnt/e/SDT/output/CORASON/modify_fabB1_fadb_nr_rep_0.7_0.7_rep_seq/pdb/103_out/103_out.pdb
Processed: /mnt/e/SDT/output/CORASON/modify_fabB1_fadb_nr_rep_0.7_0.7_rep_seq/pdb/104_out/104_out.pdb
Processed: /mnt/e/SDT/output/CORASON/modify_fabB1_fadb_nr_rep_0.7_0.7_rep_seq/pdb/105_out/105_out.pdb
Processed: /mnt/e/SDT/output/CORASON/modify_fabB1_fadb_nr_rep_0.7_0.7_rep_seq/pdb/106_out/10

# 提取分子对接结果

In [3]:
result_path = '/mnt/e/SDT/output/CORASON/modify_fabB1_fadb_nr_rep_0.7_0.7_rep_seq/mols'
import os
import csv

# 遍历每个文件夹
for folder in os.listdir(result_path):
    # 获取文件夹路径
    if os.path.isdir(os.path.join(result_path, folder)) and folder.endswith('_out'):
        folder_path = os.path.join(result_path, folder)

        # 遍历文件夹中的文件
        for file in os.listdir(folder_path):
            # 提取文件的第二行
            if file.endswith('.pdbqt'):
                file_path = os.path.join(folder_path, file)
                with open(file_path, 'r') as f:
                    lines = f.readlines()
                    if len(lines) > 1:
                        score = lines[1][20:35]
                        print(f"{folder} {file} {score}")
                        # 把结果保存到csv中
                        with open('/mnt/e/SDT/output/CORASON/modify_fabB1_fadb_nr_rep_0.7_0.7_rep_seq/result.csv', 'a') as csv_file:
                            csv_writer = csv.writer(csv_file)
                            csv_writer.writerow([folder.replace('_out',''), file.replace('_out.pdbqt',''), score])

                    
   

0_out Acetyl-CoA_out.pdbqt      -4.1      
0_out Acyl-CoA_out.pdbqt      -3.9      
0_out Butyryl-CoA_out.pdbqt      -3.8      
0_out Decanoyl-CoA_out.pdbqt      -3.6      
0_out Hexanoyl-CoA_out.pdbqt      -3.9      
0_out Hydroxy-Acetyl-CoA_out.pdbqt      -3.9      
0_out Hydroxy-Butyryl-CoA_out.pdbqt      -3.8      
0_out Hydroxy-Decanoyl-CoA_out.pdbqt      -3.5      
0_out Hydroxy-Hexanoyl-CoA_out.pdbqt      -3.9      
0_out Hydroxy-Octanoyl-CoA_out.pdbqt      -3.6      
0_out Octanoyl-CoA_out.pdbqt      -3.0      
100_out Acetyl-CoA_out.pdbqt      -3.2      
100_out Acyl-CoA_out.pdbqt      -2.7      
100_out Butyryl-CoA_out.pdbqt      -2.7      
100_out Decanoyl-CoA_out.pdbqt      -1.7      
100_out Hexanoyl-CoA_out.pdbqt      -2.7      
100_out Hydroxy-Acetyl-CoA_out.pdbqt      -3.1      
100_out Hydroxy-Butyryl-CoA_out.pdbqt      -3.4      
100_out Hydroxy-Decanoyl-CoA_out.pdbqt      -4.2      
100_out Hydroxy-Hexanoyl-CoA_out.pdbqt      -2.4      
100_out Hydroxy-Octanoyl-CoA_o

In [6]:
# 读取csv文件
import pandas as pd

df = pd.read_csv('/mnt/e/SDT/output/CORASON/modify_fabB1_fadb_nr_rep_0.7_0.7_rep_seq/result.csv',header=None)
# 重命名表头
df.columns = ['protein', 'mol', 'score']
df

,protein,mol,score
0,0,Acetyl-CoA,-4.1
1,0,Acyl-CoA,-3.9
2,0,Butyryl-CoA,-3.8
3,0,Decanoyl-CoA,-3.6
4,0,Hexanoyl-CoA,-3.9
...,...,...,...
9114,9,Hydroxy-Butyryl-CoA,-6.1
9115,9,Hydroxy-Decanoyl-CoA,-4.8
9116,9,Hydroxy-Hexanoyl-CoA,-5.8
9117,9,Hydroxy-Octanoyl-CoA,-6.0


In [21]:
pivot_df = df.pivot(index='protein', columns='mol', values='score')
pivot_df

mol,Acetyl-CoA,Acyl-CoA,Butyryl-CoA,Decanoyl-CoA,Hexanoyl-CoA,Hydroxy-Acetyl-CoA,Hydroxy-Butyryl-CoA,Hydroxy-Decanoyl-CoA,Hydroxy-Hexanoyl-CoA,Hydroxy-Octanoyl-CoA,Octanoyl-CoA
protein,,,,,,,,,,,
0,-4.1,-3.9,-3.8,-3.6,-3.9,-3.9,-3.8,-3.5,-3.9,-3.6,-3.0
1,-4.3,-3.9,-4.6,-4.2,-3.9,-3.5,-3.6,-4.2,-3.9,-3.1,-3.8
2,-4.6,-3.5,-3.8,0.9,-4.0,-2.9,-3.1,-0.1,-4.8,-2.2,-2.8
3,-4.4,-3.4,-4.1,-3.0,-3.8,-4.3,-4.0,-4.3,-3.8,-4.1,-3.3
4,-5.4,-3.1,-3.7,-4.2,-4.4,-3.9,-4.1,-3.5,-4.1,-3.4,-4.6
...,...,...,...,...,...,...,...,...,...,...,...
824,-3.5,-2.5,-3.4,-2.4,-2.6,-3.1,-3.2,-2.6,-3.1,-2.9,-2.4
825,-4.1,-3.1,-3.9,-3.4,-3.2,-4.1,-4.2,-3.9,-4.5,-4.4,-4.3
826,-4.7,-4.4,-3.5,-3.2,-3.9,-5.0,-4.4,-4.2,-4.1,-4.2,-3.5


In [22]:
# 计算每行的平均值，并创建新列'avg'
pivot_df['avg'] = pivot_df.mean(axis=1).round(1)
pivot_df['Hydroxy-Acyl-CoA'] = pivot_df['avg']

In [23]:
pivot_df

mol,Acetyl-CoA,Acyl-CoA,Butyryl-CoA,Decanoyl-CoA,Hexanoyl-CoA,Hydroxy-Acetyl-CoA,Hydroxy-Butyryl-CoA,Hydroxy-Decanoyl-CoA,Hydroxy-Hexanoyl-CoA,Hydroxy-Octanoyl-CoA,Octanoyl-CoA,avg,Hydroxy-Acyl-CoA
protein,,,,,,,,,,,,,
0,-4.1,-3.9,-3.8,-3.6,-3.9,-3.9,-3.8,-3.5,-3.9,-3.6,-3.0,-3.7,-3.7
1,-4.3,-3.9,-4.6,-4.2,-3.9,-3.5,-3.6,-4.2,-3.9,-3.1,-3.8,-3.9,-3.9
2,-4.6,-3.5,-3.8,0.9,-4.0,-2.9,-3.1,-0.1,-4.8,-2.2,-2.8,-2.8,-2.8
3,-4.4,-3.4,-4.1,-3.0,-3.8,-4.3,-4.0,-4.3,-3.8,-4.1,-3.3,-3.9,-3.9
4,-5.4,-3.1,-3.7,-4.2,-4.4,-3.9,-4.1,-3.5,-4.1,-3.4,-4.6,-4.0,-4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
824,-3.5,-2.5,-3.4,-2.4,-2.6,-3.1,-3.2,-2.6,-3.1,-2.9,-2.4,-2.9,-2.9
825,-4.1,-3.1,-3.9,-3.4,-3.2,-4.1,-4.2,-3.9,-4.5,-4.4,-4.3,-3.9,-3.9
826,-4.7,-4.4,-3.5,-3.2,-3.9,-5.0,-4.4,-4.2,-4.1,-4.2,-3.5,-4.1,-4.1


In [19]:
# 读取csv第437行

pivot_df.loc[437]

mol
Acetyl-CoA             -3.900000
Acyl-CoA               -3.300000
Butyryl-CoA            -4.300000
Decanoyl-CoA           -3.300000
Hexanoyl-CoA           -3.400000
Hydroxy-Acetyl-CoA     -4.000000
Hydroxy-Butyryl-CoA    -3.900000
Hydroxy-Decanoyl-CoA   -3.700000
Hydroxy-Hexanoyl-CoA   -4.000000
Hydroxy-Octanoyl-CoA   -3.000000
Octanoyl-CoA           -3.700000
avg                    -3.681818
Hydroxy-Acyl-CoA       -3.681818
Name: 437, dtype: float64

In [24]:
pivot_df.to_csv('/mnt/e/SDT/output/CORASON/modify_fabB1_fadb_nr_rep_0.7_0.7_rep_seq/result.csv')

# 抽取比fadb结合能高的，只保留比fadb低的，同时去除NCBI有的，去除聚类和ncbi一起的

In [1]:
# 读取csv文件
import pandas as pd

df = pd.read_csv('/mnt/e/SDT/output/CORASON/modify_fabB1_fadb_nr_rep_0.7_0.7_rep_seq/result.csv')
df

,protein,Acetyl-CoA,Acyl-CoA,Butyryl-CoA,Decanoyl-CoA,Hexanoyl-CoA,Hydroxy-Acetyl-CoA,Hydroxy-Butyryl-CoA,Hydroxy-Decanoyl-CoA,Hydroxy-Hexanoyl-CoA,Hydroxy-Octanoyl-CoA,Octanoyl-CoA,avg,Hydroxy-Acyl-CoA
0,0,-4.4,-3.8,-4.6,-4.3,-4.6,-3.9,-4.4,-4.6,-4.7,-3.6,-4.5,-4.5,-4.5
1,1,-4.3,-3.9,-4.6,-4.2,-3.9,-3.5,-3.6,-4.2,-3.9,-3.1,-3.8,-3.9,-3.9
2,2,-4.6,-3.5,-3.8,0.9,-4.0,-2.9,-3.1,-0.1,-4.8,-2.2,-2.8,-2.8,-2.8
3,3,-4.4,-3.4,-4.1,-3.0,-3.8,-4.3,-4.0,-4.3,-3.8,-4.1,-3.3,-3.9,-3.9
4,4,-5.4,-3.1,-3.7,-4.2,-4.4,-3.9,-4.1,-3.5,-4.1,-3.4,-4.6,-4.0,-4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
824,824,-3.5,-2.5,-3.4,-2.4,-2.6,-3.1,-3.2,-2.6,-3.1,-2.9,-2.4,-2.9,-2.9
825,825,-4.1,-3.1,-3.9,-3.4,-3.2,-4.1,-4.2,-3.9,-4.5,-4.4,-4.3,-3.9,-3.9
826,826,-4.7,-4.4,-3.5,-3.2,-3.9,-5.0,-4.4,-4.2,-4.1,-4.2,-3.5,-4.1,-4.1
827,827,-4.9,-4.0,-4.5,-3.6,-4.3,-5.0,-4.8,-4.3,-4.3,-4.3,-4.7,-4.4,-4.4


In [3]:
# 去除所有avg大于-4.5的行
df = df[df['avg'] <= -4.5]
df

,protein,Acetyl-CoA,Acyl-CoA,Butyryl-CoA,Decanoyl-CoA,Hexanoyl-CoA,Hydroxy-Acetyl-CoA,Hydroxy-Butyryl-CoA,Hydroxy-Decanoyl-CoA,Hydroxy-Hexanoyl-CoA,Hydroxy-Octanoyl-CoA,Octanoyl-CoA,avg,Hydroxy-Acyl-CoA
0,0,-4.4,-3.8,-4.6,-4.3,-4.6,-3.9,-4.4,-4.6,-4.7,-3.6,-4.5,-4.5,-4.5
5,5,-5.1,-4.7,-5.4,-4.1,-5.5,-5.2,-4.6,-4.9,-5.1,-5.5,-5.1,-5.0,-5.0
8,8,-6.2,-5.9,-6.0,-5.5,-5.9,-6.2,-5.9,-5.5,-5.7,-6.0,-6.4,-5.9,-5.9
9,9,-6.3,-5.3,-6.6,-5.3,-5.8,-6.2,-6.1,-4.8,-5.8,-6.0,-5.8,-5.8,-5.8
10,10,-5.6,-4.0,-5.0,-4.2,-5.5,-5.2,-5.2,-4.6,-3.8,-4.6,-5.0,-4.8,-4.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
815,815,-4.5,-4.3,-4.4,-4.1,-4.5,-4.5,-4.6,-4.1,-5.2,-4.0,-4.8,-4.5,-4.5
816,816,-5.6,-4.9,-5.4,-4.1,-4.3,-5.1,-4.5,-5.1,-5.0,-4.8,-4.3,-4.8,-4.8
819,819,-4.5,-3.9,-4.9,-5.1,-4.7,-5.1,-4.7,-3.7,-4.8,-4.6,-4.6,-4.6,-4.6
820,820,-5.0,-3.9,-5.0,-4.9,-4.9,-6.0,-5.5,-4.0,-4.3,-4.2,-4.9,-4.8,-4.8


In [4]:
# 读入匹配表
df_pair = pd.read_csv('/mnt/e/SDT/output/CORASON/modify_fabB1_fadb_nr_rep_0.7_0.7_rep_seq/prefix_mapping.csv')
df_pair

,Old Prefix,New Prefix
0,fadB,0
1,fig:666666.7133.peg.14101_Pestalotiopsis_kenya...,1
2,NCBI_ANM32233.1_hypothetical_protein_ABI59_127...,2
3,NCBI_MBL0913871.1_enoyl-CoA_hydratase-isomeras...,3
4,NCBI_MBN4066653.1_enoyl-CoA_hydratase-isomeras...,4
...,...,...
824,NCBI_MAP02243.1_hypothetical_protein_[Candidat...,824
825,NCBI_MBC6427728.1_fatty_acid_oxidation_complex...,825
826,NCBI_MCP5299546.1_enoyl-CoA_hydratase-isomeras...,826
827,NCBI_VAS23129.1_3-hydroxyacyl-CoA_dehydrogenas...,827


In [5]:
# 遍历df的行
df['ncbi'] = ''
for index, row in df.iterrows():
    # 遍历df_pair的行
    for index2, row2 in df_pair.iterrows():
        # 如果df的protein和df_pair的prefix相同
        if row['protein'] == row2['New Prefix']:
            # 将df的protein替换为df_pair的NCBI
            df.at[index, 'ncbi'] = row2['Old Prefix']

df

,protein,Acetyl-CoA,Acyl-CoA,Butyryl-CoA,Decanoyl-CoA,Hexanoyl-CoA,Hydroxy-Acetyl-CoA,Hydroxy-Butyryl-CoA,Hydroxy-Decanoyl-CoA,Hydroxy-Hexanoyl-CoA,Hydroxy-Octanoyl-CoA,Octanoyl-CoA,avg,Hydroxy-Acyl-CoA,ncbi
0,0,-4.4,-3.8,-4.6,-4.3,-4.6,-3.9,-4.4,-4.6,-4.7,-3.6,-4.5,-4.5,-4.5,fadB
5,5,-5.1,-4.7,-5.4,-4.1,-5.5,-5.2,-4.6,-4.9,-5.1,-5.5,-5.1,-5.0,-5.0,NCBI_MBT8491321.1_enoyl-CoA_hydratase-isomeras...
8,8,-6.2,-5.9,-6.0,-5.5,-5.9,-6.2,-5.9,-5.5,-5.7,-6.0,-6.4,-5.9,-5.9,NCBI_MCH8105199.1_enoyl-CoA_hydratase-isomeras...
9,9,-6.3,-5.3,-6.6,-5.3,-5.8,-6.2,-6.1,-4.8,-5.8,-6.0,-5.8,-5.8,-5.8,NCBI_PCJ66958.1_3-hydroxyacyl-CoA_dehydrogenas...
10,10,-5.6,-4.0,-5.0,-4.2,-5.5,-5.2,-5.2,-4.6,-3.8,-4.6,-5.0,-4.8,-4.8,NCBI_PYX37681.1_hypothetical_protein_DMG75_062...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
815,815,-4.5,-4.3,-4.4,-4.1,-4.5,-4.5,-4.6,-4.1,-5.2,-4.0,-4.8,-4.5,-4.5,fig:6666666.758416.peg.3671_Rhodococcus_opacus...
816,816,-5.6,-4.9,-5.4,-4.1,-4.3,-5.1,-4.5,-5.1,-5.0,-4.8,-4.3,-4.8,-4.8,fig:6666666.1012132.peg.1062_Photobacterium_pr...
819,819,-4.5,-3.9,-4.9,-5.1,-4.7,-5.1,-4.7,-3.7,-4.8,-4.6,-4.6,-4.6,-4.6,NCBI_MCP4875041.1_3-hydroxyacyl-CoA_dehydrogen...
820,820,-5.0,-3.9,-5.0,-4.9,-4.9,-6.0,-5.5,-4.0,-4.3,-4.2,-4.9,-4.8,-4.8,NCBI_TFG66930.1_fatty_acid_oxidation_complex_s...


In [6]:
# 删掉所有ncbi中有‘NCBI’的行
df = df[~df['ncbi'].str.contains('NCBI')]
df

,protein,Acetyl-CoA,Acyl-CoA,Butyryl-CoA,Decanoyl-CoA,Hexanoyl-CoA,Hydroxy-Acetyl-CoA,Hydroxy-Butyryl-CoA,Hydroxy-Decanoyl-CoA,Hydroxy-Hexanoyl-CoA,Hydroxy-Octanoyl-CoA,Octanoyl-CoA,avg,Hydroxy-Acyl-CoA,ncbi
0,0,-4.4,-3.8,-4.6,-4.3,-4.6,-3.9,-4.4,-4.6,-4.7,-3.6,-4.5,-4.5,-4.5,fadB
15,15,-6.1,-5.2,-5.1,-4.0,-5.1,-5.1,-4.8,-5.0,-5.9,-5.0,-4.0,-5.0,-5.0,fig:6666666.1014513.peg.1114_Serinibacter_salm...
16,16,-4.4,-5.4,-5.2,-4.1,-4.2,-4.6,-5.0,-5.2,-4.6,-4.4,-4.5,-4.7,-4.7,fig:6666666.1021939.peg.2578_Dichotomicrobium_...
18,18,-5.1,-3.4,-5.5,-3.4,-5.5,-4.3,-4.5,-4.2,-5.4,-4.3,-4.9,-4.6,-4.6,fig:6666666.1027448.peg.3138_Candidatus_Thiodi...
20,20,-6.3,-5.9,-5.6,-5.6,-5.9,-6.1,-6.7,-6.0,-6.3,-6.5,-5.5,-6.0,-6.0,fig:6666666.1043031.peg.2996_Ferrimonas_sedimi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
801,801,-5.7,-5.0,-5.4,-5.5,-5.8,-5.4,-5.1,-5.3,-5.2,-5.4,-5.7,-5.4,-5.4,fig:6666666.936736.peg.1684
803,803,-4.6,-4.2,-5.5,-3.3,-4.6,-4.6,-5.1,-4.5,-4.6,-4.4,-5.7,-4.6,-4.6,fig:6666666.1018818.peg.3299_Rubinisphaera_bra...
809,809,-4.2,-4.8,-5.2,-4.5,-4.5,-5.4,-4.9,-4.5,-4.5,-5.1,-3.8,-4.7,-4.7,fig:6666666.1007047.peg.876_Nitrosococcus_ocea...
815,815,-4.5,-4.3,-4.4,-4.1,-4.5,-4.5,-4.6,-4.1,-5.2,-4.0,-4.8,-4.5,-4.5,fig:6666666.758416.peg.3671_Rhodococcus_opacus...


In [10]:
for ncbi in df['ncbi']:
    print(ncbi)

fadB
fig:6666666.1014513.peg.1114_Serinibacter_salmoneus_ASM256392v1_genomic
fig:6666666.1021939.peg.2578_Dichotomicrobium_thermohalophilum_ASM355017v1_genomic
fig:6666666.1027448.peg.3138_Candidatus_Thiodiazotropha_endoloripes_ASM170895v1_genomic
fig:6666666.1043031.peg.2996_Ferrimonas_sediminicola_ASM511671v1_genomic
fig:6666666.1008003.peg.1704_Solemya_velum_gill_symbiont_ASM202185v1_genomic
fig:6666666.1010932.peg.2520_Halomonas_utahensis_ASM1990342v1_genomic
g143609.t1_Begonia_darthvaderiana_ASM2243294v1.faa
fig:6666666.888968.peg.4473_Anaeromyxobacter_sp._ASM1750v1
fig:6666666.1020528.peg.2407_Kurthia_senegalensis_ASM28559v1_genomic
fig:6666666.1045134.peg.326_Pleionea_sp._CnH1-48_ASM2412466v1_genomic
fig:6666666.971045.peg.485_Glaciecola_sp._HTCC2999_ASM15577v1
fig:6666666.1044614.peg.2587_Oceanobacillus_halotolerans_ASM1099395v1_genomic
fig:6666666.971038.peg.2031_Gemmatimonadetes_bacterium_ASM2035042v1
fig:6666666.1012081.peg.726_Photobacterium_ganghwense_ASM1908410v1_genomic


In [7]:
# 保存df的ncbi列到csv
df['ncbi'].to_csv('/mnt/e/SDT/output/CORASON/modify_fabB1_fadb_nr_rep_0.7_0.7_rep_seq/result2.0.csv')


In [8]:
# pd读取tsv文件
import pandas as pd
clust = pd.read_csv('/mnt/e/SDT/output/CORASON/fabB1_fadb_nr_rep_0.7_0.7_cluster.tsv', sep='\t',header=None)
clust

,0,1
0,g137508.t1_Stellera_chamaejasme_ASM2458632v1.faa,g137508.t1_Stellera_chamaejasme_ASM2458632v1.faa
1,g13852.t1_Stellera_chamaejasme_ASM2458632v1.faa,g13852.t1_Stellera_chamaejasme_ASM2458632v1.faa
2,g13950.t1_Stellera_chamaejasme_ASM2458632v1.faa,g13950.t1_Stellera_chamaejasme_ASM2458632v1.faa
3,g13950.t1_Stellera_chamaejasme_ASM2458632v1.faa,g61995.t1_Stellera_chamaejasme_ASM2458632v1.faa
4,g14501.t1_Stellera_chamaejasme_ASM2458632v1.faa,g14501.t1_Stellera_chamaejasme_ASM2458632v1.faa
...,...,...
891839,g56223.t1_Ficus_erecta_FER_r1.1.faa,g56223.t1_Ficus_erecta_FER_r1.1.faa
891840,g56223.t1_Ficus_erecta_FER_r1.1.faa,g36624.t1_Ficus_hispida_ASM2541302v1.faa
891841,g56225.t1_Ficus_erecta_FER_r1.1.faa,g56225.t1_Ficus_erecta_FER_r1.1.faa
891842,g56226.t1_Ficus_erecta_FER_r1.1.faa,g56226.t1_Ficus_erecta_FER_r1.1.faa


In [9]:
df = pd.read_csv('/mnt/e/SDT/output/CORASON/modify_fabB1_fadb_nr_rep_0.7_0.7_rep_seq/result3.0.csv')
# 提取以"fig:"开头的字符串中冒号后到第一个下划线之间的内容
fig_ids = []
for ncbi in df['ncbi']:
    if ncbi.startswith('fig:'):
        fig_ids.append(ncbi.split(':')[1].split('_')[0])
fig_ids

['6666666.1014513.peg.1114',
 '6666666.1021939.peg.2578',
 '6666666.1027448.peg.3138',
 '6666666.1043031.peg.2996',
 '6666666.1008003.peg.1704',
 '6666666.1010932.peg.2520',
 '6666666.1020528.peg.2407',
 '6666666.1044614.peg.2587',
 '6666666.1012081.peg.726',
 '6666666.1020420.peg.3862',
 '6666666.899274.peg.3427',
 '6666666.1012246.peg.1627',
 '6666666.1012917.peg.2852',
 '6666666.1011917.peg.1018',
 '6666666.919616.peg.2803',
 '6666666.1044207.peg.4031',
 '6666666.1029502.peg.286',
 '6666666.1018131.peg.1126',
 '6666666.1013095.peg.4561',
 '6666666.1016875.peg.3070',
 '6666666.1008929.peg.670',
 '6666666.1023255.peg.4200',
 '6666666.1045783.peg.866',
 '6666666.1030019.peg.2234',
 '6666666.1040086.peg.197',
 '6666666.1042139.peg.1375',
 '6666666.970312.peg.2249',
 '6666666.1039748.peg.1567',
 '6666666.1027290.peg.1482',
 '6666666.1041908.peg.5793',
 '6666666.1018578.peg.1621',
 '6666666.1018200.peg.3520',
 '6666666.1043239.peg.4037',
 '6666666.1028024.peg.3588',
 '6666666.1023185.peg.

In [10]:
import pandas as pd


# 创建一个集合用于记录要删除的fig_id
fig_ids_to_remove = set()

# 检查聚类中是否存在fig_id以及对应聚类中是否有以'NCBI'开头的序列
for fig_id in fig_ids:
    # 查找具体序列名称中是否存在fig_id
    matched_rows = clust[clust[1].str.contains(fig_id)]
    if not matched_rows.empty:
        # 获取包含fig_id的聚类的索引
        cluster_indices = matched_rows.index
        for idx in cluster_indices:
            # 获取该聚类的所有行
            cluster = clust.loc[clust[0] == clust.loc[idx, 0]]
            # 检查该聚类中是否存在以'NCBI'开头的序列
            if any(cluster[1].str.startswith('NCBI')):
                fig_ids_to_remove.add(fig_id)
                break  # 找到一个匹配的就可以跳出循环


# 删除原始DataFrame中对应的行
for fig_id in fig_ids_to_remove:
    df = df[~df['ncbi'].str.contains(fig_id)]

# 打印处理后的DataFrame
print(df)


    Unnamed: 0.1  Unnamed: 0  \
0              0           0   
6              6          26   
7              7          50   
12            12         131   
15            15         146   
18            18         176   
21            21         198   
26            26         266   
28            28         298   
30            30         337   
32            32         353   
36            36         436   
41            41         494   
43            43         543   
44            44         553   
47            47         643   
55            55         794   

                                                 ncbi  avg  kcatkm_Average  
0                                                fadB -4.5       58.930243  
6   fig:6666666.1010932.peg.2520_Halomonas_utahens... -4.5       62.356766  
7   g143609.t1_Begonia_darthvaderiana_ASM2243294v1... -4.7      192.707391  
12  MGYG000296234_01974_Fatty_acid_oxidation_compl... -4.5       78.695448  
15           g162173.t1_Penium_margari

In [12]:
df.to_csv('/mnt/e/SDT/output/CORASON/modify_fabB1_fadb_nr_rep_0.7_0.7_rep_seq/result3.0.csv',index=False)

In [13]:
df = pd.read_csv('/mnt/e/SDT/output/CORASON/modify_fabB1_fadb_nr_rep_0.7_0.7_rep_seq/result3.0.csv')
df

,Unnamed: 0.1,Unnamed: 0,ncbi,avg,kcatkm_Average
0,0,0,fadB,-4.5,58.930243
1,6,26,fig:6666666.1010932.peg.2520_Halomonas_utahens...,-4.5,62.356766
2,7,50,g143609.t1_Begonia_darthvaderiana_ASM2243294v1...,-4.7,192.707391
3,12,131,MGYG000296234_01974_Fatty_acid_oxidation_compl...,-4.5,78.695448
4,15,146,g162173.t1_Penium_margaritaceum_V1.0.faa,-4.5,64.380682
5,18,176,g56484.t1_Saccharum_hybrid_CTBE_SP803280_v1.0.faa,-5.0,81.672978
6,21,198,g344048.t1_Streptochaeta_angustifolia_ISU_Sang...,-4.8,58.934070
7,26,266,g212398.t1_Papaver_nudicaule_Yrk_Pnu_10X_1.faa,-5.1,357.729727
8,28,298,MGYG000296939_02807_Fatty_acid_oxidation_compl...,-4.6,59.504610
9,30,337,fig:6666666.1045783.peg.866_Siccirubricoccus_s...,-5.1,60.090172


# 读入kcatkm 预测结果，将对接结果和预测结果合并


In [1]:
import pandas as pd
# 读入kcatkm预测结果
df = pd.read_csv('/mnt/e/UniKP/Kcat_Km/pre_corason_all.csv').head(1650)
kcatkm = df[['Average','pro']]
kcatkm


,Average,pro
0,421.066504,g223962.t1_Papaver_nudicaule_Yrk_Pnu_10X_1.faa
1,357.729727,g212398.t1_Papaver_nudicaule_Yrk_Pnu_10X_1.faa
2,320.788675,g155135.t1_Verbascum_thapsus_ASM2856639v1.faa
3,320.739716,Vfaba.Hedin2.R1.4g092400.1
4,309.269464,NCBI_KAF1858496.1_hypothetical_protein_Lal_000...
...,...,...
1645,58.630266,NCBI_WP_168875591.1_enoyl-CoA_hydratase/isomer...
1646,58.622736,NCBI_NTW97953.1_3-hydroxyacyl-CoA_dehydrogenas...
1647,58.609049,NCBI_MBC7982880.1_enoyl-CoA_hydratase/isomeras...
1648,58.606468,fig:6666666.1023741.peg.168_Halopseudomonas_sa...


In [2]:
df = pd.read_csv('/mnt/e/SDT/output/CORASON/modify_fabB1_fadb_nr_rep_0.7_0.7_rep_seq/result3.0.csv')
df

,Unnamed: 0,ncbi,avg
0,0,fadB,-4.5
1,15,fig:6666666.1014513.peg.1114_Serinibacter_salm...,-5.0
2,16,fig:6666666.1021939.peg.2578_Dichotomicrobium_...,-4.7
3,18,fig:6666666.1027448.peg.3138_Candidatus_Thiodi...,-4.6
4,20,fig:6666666.1043031.peg.2996_Ferrimonas_sedimi...,-6.0
5,25,fig:6666666.1008003.peg.1704_Solemya_velum_gil...,-4.5
6,26,fig:6666666.1010932.peg.2520_Halomonas_utahens...,-4.5
7,50,g143609.t1_Begonia_darthvaderiana_ASM2243294v1...,-4.7
8,61,fig:6666666.1020528.peg.2407_Kurthia_senegalen...,-4.8
9,87,fig:6666666.1044614.peg.2587_Oceanobacillus_ha...,-5.1


In [5]:
import pandas as pd

# 假设你的DataFrame已经读取并存储在变量df和kcatkm中

# 提取以"fig:"开头的字符串中冒号后到第一个下划线之间的内容
df['fig_id'] = df['ncbi'].apply(lambda x: x.split(':')[1].split('_')[0] if x.startswith('fig:') else x)

# 创建一个字典来存储fig_id和kcatkm['Average']的对应关系
fig_id_to_average = {}

# 遍历kcatkm，检查每个pro是否包含在fig_id中
for index, row in kcatkm.iterrows():
    for fig_id in df['fig_id']:
        if fig_id in row['pro']:
            # 保留小数点2位
            fig_id_to_average[fig_id] = row['Average']

# 添加一个新的列，用于存储kcatkm['Average']的值
df['kcatkm_Average'] = df['fig_id'].map(fig_id_to_average)

# 打印处理后的DataFrame
print(df)

# 如果不再需要fig_id列，可以删除
df.drop(columns=['fig_id'], inplace=True)

        

    Unnamed: 0                                               ncbi  avg  \
0            0                                               fadB -4.5   
1           15  fig:6666666.1014513.peg.1114_Serinibacter_salm... -5.0   
2           16  fig:6666666.1021939.peg.2578_Dichotomicrobium_... -4.7   
3           18  fig:6666666.1027448.peg.3138_Candidatus_Thiodi... -4.6   
4           20  fig:6666666.1043031.peg.2996_Ferrimonas_sedimi... -6.0   
5           25  fig:6666666.1008003.peg.1704_Solemya_velum_gil... -4.5   
6           26  fig:6666666.1010932.peg.2520_Halomonas_utahens... -4.5   
7           50  g143609.t1_Begonia_darthvaderiana_ASM2243294v1... -4.7   
8           61  fig:6666666.1020528.peg.2407_Kurthia_senegalen... -4.8   
9           87  fig:6666666.1044614.peg.2587_Oceanobacillus_ha... -5.1   
10          96  fig:6666666.1012081.peg.726_Photobacterium_gan... -4.5   
11          97  fig:6666666.1020420.peg.3862_Marinobacterium_l... -5.0   
12         131  MGYG000296234_01974_Fa

In [6]:
df.to_csv('/mnt/e/SDT/output/CORASON/modify_fabB1_fadb_nr_rep_0.7_0.7_rep_seq/result3.0.csv')

In [4]:
import pandas as pd
df = pd.read_csv('/mnt/e/SDT/output/CORASON/modify_fabB1_fadb_nr_rep_0.7_0.7_rep_seq/result3.0.csv')
df.sort_values(by=['kcatkm_Average'], ascending=False, inplace=True)
df.drop(columns=['Unnamed: 0.1','Unnamed: 0'], inplace=True)

In [5]:
df

,ncbi,avg,kcatkm_Average
26,g212398.t1_Papaver_nudicaule_Yrk_Pnu_10X_1.faa,-5.1,357.729727
7,g143609.t1_Begonia_darthvaderiana_ASM2243294v1...,-4.7,192.707391
43,g12868.t1_Phalaenopsis_hybrid_cultivar_ASM2079...,-4.6,126.918039
51,fig:6666666.1027300.peg.1460_Halofilum_ochrace...,-4.6,91.141567
4,fig:6666666.1043031.peg.2996_Ferrimonas_sedimi...,-6.0,87.584996
55,fig:6666666.970809.peg.7932_Amphritea_spongico...,-5.1,86.413088
13,fig:6666666.899274.peg.3427_Syntrophomonadacea...,-6.1,85.937170
33,fig:6666666.1040086.peg.197_Ningiella_ruwaisen...,-5.7,82.815395
18,g56484.t1_Saccharum_hybrid_CTBE_SP803280_v1.0.faa,-5.0,81.672978
27,fig:6666666.1008929.peg.670_Thiothrix_unzii_AS...,-5.0,81.118128


In [1]:
import pandas as pd
df = pd.read_csv('/mnt/e/SDT/output/CORASON/modify_fabB1_fadb_nr_rep_0.7_0.7_rep_seq/result3.0.csv')
df

,Unnamed: 0.1,Unnamed: 0,ncbi,avg,kcatkm_Average
0,0,0,fadB,-4.5,58.930243
1,7,50,g143609.t1_Begonia_darthvaderiana_ASM2243294v1...,-4.7,192.707391
2,15,146,g162173.t1_Penium_margaritaceum_V1.0.faa,-4.5,64.380682
3,18,176,g56484.t1_Saccharum_hybrid_CTBE_SP803280_v1.0.faa,-5.0,81.672978
4,21,198,g344048.t1_Streptochaeta_angustifolia_ISU_Sang...,-4.8,58.934070
5,26,266,g212398.t1_Papaver_nudicaule_Yrk_Pnu_10X_1.faa,-5.1,357.729727
6,28,298,MGYG000296939_02807_Fatty_acid_oxidation_compl...,-4.6,59.504610
7,30,337,fig:6666666.1045783.peg.866_Siccirubricoccus_s...,-5.1,60.090172
8,36,436,sefadB,-5.1,59.596168
9,43,543,g12868.t1_Phalaenopsis_hybrid_cultivar_ASM2079...,-4.6,126.918039


In [3]:
# 用bio读取fasta文件
from Bio import SeqIO

# 读取fasta文件
fasta_file = '/mnt/e/SDT/output/CORASON/modify_fabB1_fadb_nr_rep_0.7_0.7_rep_seq/逐梦演艺圈.fasta'

# 依次打印序列名称和长度
for record in SeqIO.parse(fasta_file, "fasta"):
    print(record.id, len(record))


g143609.t1_Begonia_darthvaderiana_ASM2243294v1.faa 626
g162173.t1_Penium_margaritaceum_V1.0.faa 659
g56484.t1_Saccharum_hybrid_CTBE_SP803280_v1.0.faa 758
g344048.t1_Streptochaeta_angustifolia_ISU_Sangustifolia_v1.faa 512
g212398.t1_Papaver_nudicaule_Yrk_Pnu_10X_1.faa 947
MGYG000296939_02807_Fatty_acid_oxidation_complex_subunit_alpha_marine 741
fig:6666666.1045783.peg.866_Siccirubricoccus_sp._G192_ASM1908407v1_genomic 628
g12868.t1_Phalaenopsis_hybrid_cultivar_ASM207920v1.faa 784
scaffold-PQED-2053540-Gloeochaete_wittrockiana 616
g62632.t1_Coffea_canephora_AUK_PRJEB4211_v1 971
